# https://hh.ru/ web scraping

## I am going to collect the information about all data analyst vacations in Moscow (as an example) from the biggest headhunter website in Russia. But the final code will be able to collect the information about any vacations

In [3]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

import pandas as pd

In [121]:
s = Service("./chromedriver")
browser = Chrome(service = s)
url = "https://hh.ru/"

browser.get(url)

### looking for the input field and searching vacancies

In [122]:
input_field = browser.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div[3]/div/div/form/div/div[1]/div/input")
input_field

<selenium.webdriver.remote.webelement.WebElement (session="2eea7e33de377d3522c0b1fc604820fb", element="f4a33363-523b-4e29-966e-87de9d2c644d")>

In [123]:
vacancy = str(input())
input_field.send_keys(vacancy)


танцор


In [124]:
search_but = browser.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div[3]/div/div/form/div/div[2]/button")
search_but.click()

### data collecting from the first page

In [125]:
soup = BeautifulSoup(browser.page_source, 'lxml')
#soup

In [126]:
cards = soup.findAll("div", class_ = "vacancy-serp-item")

In [127]:
for card in cards:
    name = card.find("a", class_ = "bloko-link").text.strip()
    name

In [128]:
spans = card.findAll("span", class_ = "bloko-header-section-3 bloko-header-section-3_lite")
salary1 = spans[1].text.strip().split()
salary = "".join(salary1)
salary

'от35000руб.'

In [129]:
link = card.find("a", class_ = "bloko-link").get("href")
link

'https://hh.ru/vacancy/50814676?query=%D1%82%D0%B0%D0%BD%D1%86%D0%BE%D1%80'

In [130]:
company = card.find("a", class_ = "bloko-link bloko-link_kind-secondary").text.strip()
company

'Танцевальная мастерская, Танцевальная школа'

### page switching (clicking "next" button)

### this button has tag "a" and class "bloko-button". It is always the last element on the page with such parameters, so we will use 
```python
browser.find.elements(...)[-1]
```

In [133]:
try:
    nextbutton = browser.find_element(By.CSS_SELECTOR, "[data-qa='pager-next']")


SyntaxError: unexpected EOF while parsing (<ipython-input-133-ee53c59a40ab>, line 2)

# Full code

In [135]:
vacancy_name = str(input())

s = Service("./chromedriver")
browser = Chrome(service = s)
url = "https://hh.ru/"

data = []

browser.get(url)
sleep(5)

input_field = browser.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div[3]/div/div/form/div/div[1]/div/input")
input_field.send_keys(vacancy_name)
browser.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/div[1]/div[3]/div/div/form/div/div[2]/button").click()

sleep(5)

while True:
    sleep(5)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    cards = soup.findAll("div", class_ = "vacancy-serp-item")
    
    for card in cards:
        try:
            name = card.find("a", class_ = "bloko-link").text.strip()
        except:
            name = "-"
        try:
            spans = card.findAll("span", class_ = "bloko-header-section-3 bloko-header-section-3_lite")
            salary1 = spans[1].text.strip().split()
            salary = "".join(salary1)
            salary
        except:
            salary = "-"
        try:
            company = card.find("a", class_ = "bloko-link bloko-link_kind-secondary").text.strip()
        except:
            company = "-"
        try:
            link = card.find("a", class_ = "bloko-link").get("href")
        except:
            link = "-"
        data.append([name, salary, company, link])
    print(len(data), "cards collected")
        
    try:
        browser.find_element(By.CSS_SELECTOR, "[data-qa='pager-next']").click()
    except:
        break


data analyst
61 cards collected
111 cards collected
161 cards collected
211 cards collected
261 cards collected
311 cards collected
361 cards collected
411 cards collected
461 cards collected
511 cards collected
561 cards collected
611 cards collected
661 cards collected
711 cards collected
761 cards collected
811 cards collected
861 cards collected
911 cards collected
961 cards collected
1011 cards collected
1061 cards collected
1111 cards collected
1161 cards collected
1211 cards collected
1261 cards collected
1311 cards collected
1361 cards collected
1411 cards collected
1461 cards collected
1511 cards collected
1561 cards collected
1611 cards collected
1661 cards collected
1711 cards collected
1761 cards collected
1811 cards collected
1861 cards collected
1911 cards collected
1961 cards collected
2011 cards collected
2061 cards collected


In [137]:
headers = ["name", "salary", "company", "link"]
df = pd.DataFrame(data, columns = headers)
df.to_csv("./hhru.csv", sep = ";", encoding = 'utf8')